In [5]:
import numpy as np
import json
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, GRU, Dropout, Dense


def predict_next_day_values(data_path, get_value: str, epochs=50):
    # Load data from JSON file
    with open(data_path) as f:
        data = json.load(f)

    # Extract features and target
    X = np.column_stack((data["o"], data["h"], data["l"], data["c"], data["v"]))
    y = np.array(data[get_value])

    # Normalize data
    scaler_X = MinMaxScaler()
    scaler_y = MinMaxScaler()
    X_scaled = scaler_X.fit_transform(X)
    y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))

    # Reshape data for LSTM input (samples, timesteps, features)
    X_scaled = X_scaled.reshape(X_scaled.shape[0], X_scaled.shape[1], 1)

    # Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y_scaled, test_size=0.3, random_state=42
    )

    # Build the model
    model = Sequential()
    model.add(
        Bidirectional(
            GRU(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1))
        )
    )
    model.add(Dropout(0.2))
    model.add(Bidirectional(GRU(units=50)))
    model.add(Dropout(0.2))
    model.add(Dense(32, kernel_initializer="uniform", activation="relu"))
    model.add(Dense(1, kernel_initializer="uniform", activation="linear"))
    model.compile(optimizer="adam", loss="mean_squared_error")

    # Train the model
    model.fit(X_train, y_train, epochs=epochs, batch_size=32, verbose=1)

    # Predict tomorrow's value
    last_data_point = X_scaled[-1].reshape(1, X_scaled.shape[1], 1)
    predicted_value_scaled = model.predict(last_data_point)
    predicted_value = scaler_y.inverse_transform(predicted_value_scaled)

    return predicted_value


# Define data file path
data_path = "data.json"

# Define values to predict
values_to_predict = ["c"]

# Predict for each value
for value in values_to_predict:
    predicted_value = predict_next_day_values(data_path, value, epochs=500)
    print(f"Predicted {value} for tomorrow:", predicted_value)

Epoch 1/500
126/126 [==============================] - 5s 18ms/step - loss: 0.0230
Epoch 2/500
126/126 [==============================] - 2s 16ms/step - loss: 3.7025e-04
Epoch 3/500
126/126 [==============================] - 2s 16ms/step - loss: 2.1670e-04
Epoch 4/500
126/126 [==============================] - 2s 16ms/step - loss: 1.6792e-04
Epoch 5/500
126/126 [==============================] - 2s 16ms/step - loss: 1.1328e-04
Epoch 6/500
126/126 [==============================] - 2s 16ms/step - loss: 8.1857e-05
Epoch 7/500
126/126 [==============================] - 2s 16ms/step - loss: 6.5679e-05
Epoch 8/500
126/126 [==============================] - 2s 16ms/step - loss: 5.1389e-05
Epoch 9/500
126/126 [==============================] - 2s 16ms/step - loss: 4.6521e-05
Epoch 10/500
126/126 [==============================] - 2s 16ms/step - loss: 6.2246e-05
Epoch 11/500
126/126 [==============================] - 2s 16ms/step - loss: 1.0902e-04
Epoch 12/500
126/126 [=======================